In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/train.csv')
test = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/test.csv')
sub = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/sample_submission.csv')
age = pd.read_csv('C:/Users/user/LikeLion_AI/data/CSV/DACON_Parking_Demand/age_gender_info.csv')

train.shape, test.shape, sub.shape, age.shape

((2952, 15), (1022, 14), (150, 2), (16, 23))

### 컬럼 확인

In [4]:
train.columns, test.columns

(Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
        '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
        '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
       dtype='object'),
 Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
        '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
        '도보 10분거리 내 버스정류장 수', '단지내주차면수'],
       dtype='object'))

### head 확인

In [6]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


### 범주형 변수 몇 개만 확인

In [10]:
print(train['임대건물구분'].value_counts())
print(train['지역'].value_counts())

아파트    2390
상가      562
Name: 임대건물구분, dtype: int64
경기도        621
경상남도       369
부산광역시      328
대전광역시      266
강원도        187
충청북도       163
충청남도       158
전라남도       151
광주광역시      141
경상북도       119
전라북도       109
대구광역시      102
제주특별자치도     91
서울특별시       64
세종특별자치시     47
울산광역시       36
Name: 지역, dtype: int64


### 라벨 인코딩

In [18]:
from sklearn.preprocessing import LabelEncoder

train_1 = train.copy()

encoder = LabelEncoder()
encoder.fit(train['임대건물구분'])
train_1['임대건물구분'] = encoder.transform(train['임대건물구분'])

encoder.fit(train_1['지역'])
train_1['지역'] = encoder.transform(train['지역'])

encoder.fit(train_1['공급유형'])
train_1['공급유형'] = encoder.transform(train['공급유형'])
train_1['공급유형'].value_counts()

5    1758
7     562
9     213
1     205
6     152
2      31
4      12
8       9
0       7
3       3
Name: 공급유형, dtype: int64

In [19]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   int32  
 3   지역                            2952 non-null   int32  
 4   공급유형                          2952 non-null   int32  
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

### 모델 구성 및 예측

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

sel = ['총세대수', '전용면적', '임대건물구분', '지역', '공급유형', '전용면적별세대수']

X = train_1[sel]
y = train_1['등록차량수']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)
pred

array([ 799.08633276,  924.05373924,  794.49680274,  935.29435669,
        718.886069  ,  -40.64909845,   92.72745026, -173.27894158,
        607.595613  ,  603.29916284,  643.12904528,  653.64852343,
        824.43173629,  833.70696259,  679.90359563,  477.44309924,
        961.89942264,  662.71499712,  290.40876959,  583.54189218,
        -68.83852422,  649.3258386 ,  746.85948506,  713.89769443,
          5.98504325,  694.81509749,  724.33605276,  823.96314359,
        573.98504671,  804.20814317,  690.5973213 , 1073.13714523,
        633.53219335,  551.09738376,  -66.24386857,  603.65296854,
        738.51094684,  676.13676613,  659.17525999, 1291.52770016,
        410.60360652,  486.67777356,   16.84832946,  810.49589012,
        263.36183605,  927.35081293, 1132.87017809,  466.94599463,
        701.89768071,  465.5488566 ,  587.89528498,  616.32229721,
        581.5485822 ,  849.19747352,  503.56396401, 1173.73708338,
        592.69144746, -174.26796326,  724.73473434,  603.44146

### 모델 평가

In [24]:
MAE = abs(y_test - pred).sum() / len(y_test)
MSE = ((y_test - pred)**2).sum() / len(y_test)
RMSE = np.sqrt(((y_test - pred)**2).sum() / len(y_test))

print('MAE : {:.3f}, MSE : {:.3f}, RMSE : {:.3f}'.format(MAE, MSE, RMSE))

MAE : 208.006, MSE : 80842.770, RMSE : 284.329


### 계속 진행중 ...